In [1]:
import pandas as pd
import numpy as np

In [2]:
FechaInicio = '20/08/2023'
FechaFinal = '20/09/2023'

In [3]:
# CALCULOS PCTJE DE ACIERTOS
FechaComienzo = FechaInicio.replace('/', '-')

FechaTermino = FechaFinal.replace('/', '-')

FileName_DataObservada = f"calidad-aire-dia-hora_Sierra_Gorda_{FechaComienzo}_a_{FechaTermino}.csv"

FileName_DataModelada = f"calidad-aire-dia-hora_Sierra_Gorda-PRO_{FechaComienzo}_a_{FechaTermino}.csv"

DataObservada = pd.read_csv(f'../Observados/ScrapedGeologger/{FileName_DataObservada}')

DataModelada = pd.read_csv(f'../Modelados/ScrapedGeologger/{FileName_DataModelada}')

DataObservada['Hora'] = DataObservada['Hora'].astype(str) + ':00'

DataObservada['Fecha_y_Hora'] = pd.to_datetime(DataObservada['Fecha'] + ' ' + DataObservada['Hora'], format='%d/%m/%Y %H:%M')

DataModelada['Hora'] = DataModelada['Hora'].astype(str) + ':00'

DataModelada['Fecha_y_Hora'] = pd.to_datetime(DataModelada['Fecha'] + ' ' + DataModelada['Hora'], format='%d/%m/%Y %H:%M')

df_merged = pd.merge(DataObservada[['Fecha_y_Hora', 'MP10 (µg/m³N)']], DataModelada[['Fecha_y_Hora', 'MP10 (µg/m³N)']], on='Fecha_y_Hora', suffixes=('_obs', '_pro'))

df_merged['Bueno'] = np.where(
    (((df_merged['MP10 (µg/m³N)_obs'] >= 0) & (df_merged['MP10 (µg/m³N)_obs'] <= 130) & 
      (df_merged['MP10 (µg/m³N)_pro'] >= 0) & (df_merged['MP10 (µg/m³N)_pro'] <= 130)) | 
     ((df_merged['MP10 (µg/m³N)_obs'] < 0) | (df_merged['MP10 (µg/m³N)_obs'] > 130) & 
      (df_merged['MP10 (µg/m³N)_pro'] < 0) | (df_merged['MP10 (µg/m³N)_pro'] > 130))), 1, 0)

df_merged['Alerta1'] = np.where(
    (((df_merged['MP10 (µg/m³N)_obs'] > 130) & (df_merged['MP10 (µg/m³N)_obs'] <= 180) & 
      (df_merged['MP10 (µg/m³N)_pro'] > 130) & (df_merged['MP10 (µg/m³N)_pro'] <= 180)) | 
     ((df_merged['MP10 (µg/m³N)_obs'] <= 130) | (df_merged['MP10 (µg/m³N)_obs'] > 180) & 
      (df_merged['MP10 (µg/m³N)_pro'] <= 130) | (df_merged['MP10 (µg/m³N)_pro'] > 180))), 1, 0)

df_merged['Alerta2'] = np.where(
    ((df_merged['MP10 (µg/m³N)_obs'] > 180) & (df_merged['MP10 (µg/m³N)_pro'] > 180)) | 
    ((df_merged['MP10 (µg/m³N)_obs'] < 180) & (df_merged['MP10 (µg/m³N)_pro'] < 180)), 1, 0)

pct_bueno = df_merged['Bueno'].sum() / len(df_merged)
pct_alerta1 = df_merged['Alerta1'].sum() / len(df_merged)
pct_alerta2 = df_merged['Alerta2'].sum() / len(df_merged)

promedio = np.mean([pct_bueno, pct_alerta1, pct_alerta2])

In [4]:
# RESULTADOS
print(f'Porcentaje Bueno: {pct_bueno}')
print(f'Porcentaje Alerta1: {pct_alerta1}')
print(f'Porcentaje Alerta2: {pct_alerta2}')

print(f'Porcentaje Promedio: {promedio}')

Porcentaje Bueno: 0.9051724137931034
Porcentaje Alerta1: 0.9252873563218391
Porcentaje Alerta2: 0.7844827586206896
Porcentaje Promedio: 0.8716475095785441
